In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x3845f80
Welcome to JupyROOT 6.30/06


In [2]:
%jsroot on

In [3]:
ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.ildStyle.SetOptStat(0)
# for nicer 2d colz
ROOT.ildStyle.SetPalette(ROOT.kBird)
ROOT.ildStyle.SetNumberContours(256)

In [5]:
%%cpp
using namespace ROOT::VecOps;

In [6]:
prefix = "data/k4gen/whizard_new"

dfs = {}

dfs["sw"] = ROOT.RDataFrame("events", (f"{prefix}/enuqq_sw_e.edm4hep.root", f"{prefix}/enuqq_sw_p.edm4hep.root"))
dfs["all"] = ROOT.RDataFrame("events", f"{prefix}/enuqq_all.edm4hep.root")
# df = ROOT.RDataFrame("events", "data/truejet/test/sw_sl/nano.root")
# df = ROOT.RDataFrame("events", "data/truejet/test/ww_sl/nano.root")

In [7]:
# these files are very simple, they have the Ws explicitly and there is only one stable e/p
for df in dfs:
    dfs[df] = dfs[df].Define("l_MC_idx", "ArgMax((abs(MCParticle.PDG) == 11 || abs(MCParticle.PDG) == 13) && MCParticle.generatorStatus == 1)")
    dfs[df] = dfs[df].Define("nu_MC_idx", "ArgMax((abs(MCParticle.PDG) == 12 || abs(MCParticle.PDG) == 14) && MCParticle.generatorStatus == 1)")

    dfs[df] = dfs[df].Define("l_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticle.momentum.x[l_MC_idx], MCParticle.momentum.y[l_MC_idx], MCParticle.momentum.z[l_MC_idx], MCParticle.mass[l_MC_idx])")
    dfs[df] = dfs[df].Define("nu_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticle.momentum.x[nu_MC_idx], MCParticle.momentum.y[nu_MC_idx], MCParticle.momentum.z[nu_MC_idx], MCParticle.mass[nu_MC_idx])")

    dfs[df] = dfs[df].Define("l_pdg", "MCParticle.PDG[l_MC_idx]")
    dfs[df] = dfs[df].Define("nu_pdg", "MCParticle.PDG[nu_MC_idx]")

In [8]:

for df in dfs:
    dfs[df] = dfs[df].Define("l_MC_lvec_e", "l_MC_lvec.energy()")
    dfs[df] = dfs[df].Define("l_MC_lvec_theta", "l_MC_lvec.theta()")
    dfs[df] = dfs[df].Define("l_MC_lvec_cosTheta", "cos(l_MC_lvec_theta)")

    dfs[df] = dfs[df].Define("l_MC_lvec_pt", "l_MC_lvec.pt()")
    dfs[df] = dfs[df].Define("l_MC_lvec_pz", "l_MC_lvec.pz()")

    dfs[df] = dfs[df].Define("nu_MC_lvec_e", "nu_MC_lvec.energy()")

    dfs[df] = dfs[df].Define("lnu_MC_m", "(l_MC_lvec + nu_MC_lvec).mass()")

    # FIXME: no crossing angle!
    dfs[df] = dfs[df].Define("l_beam_lvec", "ROOT::Math::PxPyPzMVector(0, 0, std::copysign(125., l_pdg), 0.0005109968178652498)")
    dfs[df] = dfs[df].Define("q2", "(l_beam_lvec - nu_MC_lvec).M2()")

In [9]:
dfs["all"] = dfs["all"].Define("weight", "1.0")
# need to multiply with BR(W -> ud) ~ 1/3 and treat it like it has the same xsec as all
xsec_sw = 1.6810066E+04 # [fb]
xsec_all = 6.7000588E+02 # [fb]
lumi_sw = 5.9419E+00 # [fb]
lumi_all = 7.4644E+01 # [fb]
dfs["sw"] = dfs["sw"].Define("weight", f"3.3339791e-01 * {xsec_all / xsec_sw} * {lumi_all / lumi_sw}")

In [10]:
h_sw_m = dfs["sw"].Histo1D(("", ";m_{l+#nu} [GeV]", 50, 0., 200.), "lnu_MC_m", "weight")
h_all_m = dfs["all"].Histo1D(("", ";m_{l+#nu} [GeV]", 50, 0., 200.), "lnu_MC_m", "weight")

h_sw_q2 = dfs["sw"].Histo1D(("", ";q^{2} [GeV^{2}]", 35, -60e3, 1e3), "q2", "weight")
h_all_q2 = dfs["all"].Histo1D(("", ";q^{2} [GeV^{2}]", 35, -60e3, 1e3), "q2", "weight")

h_2d_sw_q2_m = dfs["sw"].Histo2D(("", ";q^{2} [GeV^{2}];m_{l,#nu} [GeV]", 70, -60e3, 1e3, 100, 0., 200.), "q2", "lnu_MC_m", "weight")
h_2d_all_q2_m = dfs["all"].Histo2D(("", ";q^{2} [GeV^{2}];m_{l,#nu} [GeV]", 70, -60e3, 1e3, 100, 0., 200.), "q2", "lnu_MC_m", "weight")

In [11]:
c_lnu_m_sw = ROOT.TCanvas()
h_all_m.Draw("hist")
h_sw_m.SetMarkerColor(ROOT.kRed)
h_sw_m.SetLineColor(ROOT.kRed)
h_sw_m.Draw("same hist")
c_lnu_m_sw.Draw()

c_lnu_m_ratio = ROOT.TCanvas()
lnu_m_ratio = ROOT.TRatioPlot(h_all_m.GetPtr(), h_sw_m.GetPtr(), "diff")
lnu_m_ratio.Draw()
c_lnu_m_ratio.Draw()

c_q2_ratio = ROOT.TCanvas()
q2_ratio = ROOT.TRatioPlot(h_all_q2.GetPtr(), h_sw_q2.GetPtr(), "diff")
q2_ratio.Draw()
c_q2_ratio.Draw()

c_2d_sw_q2_m = ROOT.TCanvas()
h_2d_sw_q2_m.Draw("colz0")
c_2d_sw_q2_m.SetLogz()
c_2d_sw_q2_m.Draw()

c_2d_all_q2_m = ROOT.TCanvas()
h_2d_all_q2_m.Draw("colz0")
c_2d_all_q2_m.SetLogz()
c_2d_all_q2_m.Draw()